# Install and Load Libraries

In [11]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 596, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 596 (delta 128), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (596/596), 195.77 KiB | 6.31 MiB/s, done.
Resolving deltas: 100% (302/302), done.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.13 environment at: /usr
Resolved 173 packages in 3.11s
 Downloaded cuproj-cu12
 Downloaded rmm-cu12
 Downloaded cudf-cu12
 Downloaded shapely
 Downloaded datashader
 Downloaded libkvikio-cu12
 Downloaded pylibcugraph-cu12
 Downloaded ucx-py-cu12
 Downloaded dask
 Downloaded bokeh
 Downloaded cuspatial-cu12
 Downloaded librmm-cu12
 Downloaded cugraph-cu12
 Downloaded cuml-cu12
 Downloaded cucim-cu12
 Downloaded libcuspatial-cu12
 Downloaded pylibcudf-cu12
 Downloaded libraft-cu12
 Downloaded libcudf-cu12
 Downloaded raft-dask-cu12
 Downloaded libcuml-cu12
 Downloaded libcuvs

In [12]:
!pip install kaggle --quiet

In [13]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle datasets download -d sobhanmoosavi/us-accidents

Dataset URL: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents
License(s): CC-BY-NC-SA-4.0
 99% 645M/653M [00:07<00:00, 216MB/s]
100% 653M/653M [00:07<00:00, 92.5MB/s]


In [15]:
!unzip us-accidents.zip

Archive:  us-accidents.zip
  inflating: US_Accidents_March23.csv  


In [16]:
import dask.dataframe as dd

df = dd.read_csv("/content/US_Accidents_March23.csv")

In [17]:
import cudf
cudf.__version__

'25.02.01'

# Data Extraction

In [18]:
import cudf

df = cudf.read_csv("/content/US_Accidents_March23.csv")

In [19]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype
---  ------                 -----
 0   ID                     object
 1   Source                 object
 2   Severity               int64
 3   Start_Time             object
 4   End_Time               object
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object
 11  Street                 object
 12  City                   object
 13  County                 object
 14  State                  object
 15  Zipcode                object
 16  Country                object
 17  Timezone               object
 18  Airport_Code           object
 19  Weather_Timestamp      object
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)            float64
 23  

In [20]:
df.describe()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,7.728394e+06,7.728394e+06,7.728394e+06,4.325632e+06,4.325632e+06,7.728394e+06,7.564541e+06,5.729375e+06,7.554250e+06,7.587715e+06,7.551296e+06,7.157161e+06,5.524808e+06
mean,2.212384e+00,3.620119e+01,-9.470255e+01,3.626183e+01,-9.572557e+01,5.618420e-01,6.166329e+01,5.825105e+01,6.483104e+01,2.953899e+01,9.090376e+00,7.685490e+00,8.407000e-03
std,4.875310e-01,5.076079e+00,1.739176e+01,5.272905e+00,1.810793e+01,1.776811e+00,1.901365e+01,2.238983e+01,2.282097e+01,1.006190e+00,2.688316e+00,5.424983e+00,1.102250e-01
min,1.000000e+00,2.455480e+01,-1.246238e+02,2.456601e+01,-1.245457e+02,0.000000e+00,-8.900000e+01,-8.900000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,3.339963e+01,-1.172194e+02,3.346207e+01,-1.177543e+02,0.000000e+00,4.900000e+01,4.300000e+01,4.800000e+01,2.937000e+01,1.000000e+01,4.600000e+00,0.000000e+00
50%,2.000000e+00,3.582397e+01,-8.776662e+01,3.618350e+01,-8.802789e+01,3.000000e-02,6.400000e+01,6.200000e+01,6.700000e+01,2.986000e+01,1.000000e+01,7.000000e+00,0.000000e+00
75%,2.000000e+00,4.008496e+01,-8.035368e+01,4.017892e+01,-8.024709e+01,4.640000e-01,7.600000e+01,7.500000e+01,8.400000e+01,3.003000e+01,1.000000e+01,1.040000e+01,0.000000e+00
max,4.000000e+00,4.900220e+01,-6.711317e+01,4.907500e+01,-6.710924e+01,4.417500e+02,2.070000e+02,2.070000e+02,1.000000e+02,5.863000e+01,1.400000e+02,1.087000e+03,3.647000e+01


# PreProcessing

In [63]:
import pandas as pd
import cudf

df = cudf.read_csv("/content/US_Accidents_March23.csv")



print("🧠 حجم اولیه دیتافریم (GB):", df.memory_usage(deep=True).sum() / 1e9)


🧠 حجم اولیه دیتافریم (GB): 3.242436959


In [64]:

if 'Country' in df.columns:
    df['Country'] = (df['Country'] == 'US').astype('uint8')


In [65]:
for col in df.select_dtypes(include=['int64', 'float64']).columns:
    df[col] = pd.to_numeric(df[col].to_pandas(), downcast='unsigned' if df[col].min() >= 0 else 'float')

In [66]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

label_encoders = {}

for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df_col_pandas = df[col].astype(str).fillna("missing").to_pandas()
    encoded_col_pandas = le.fit_transform(df_col_pandas)
    df[col] = cudf.Series(encoded_col_pandas)
    label_encoders[col] = le

In [67]:
cols_to_keep_mask = df.isnull().mean() < 0.9
cols_to_keep = cols_to_keep_mask[cols_to_keep_mask].index.to_arrow().to_pylist()
df = df[cols_to_keep]

In [68]:
df.dropna(subset=['Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng'], inplace=True)

In [69]:
df['Start_Date'] = pd.to_datetime(df['Start_Time'].to_pandas()).dt.date.astype(str)
df.drop(['Start_Time', 'End_Time'], axis=1, inplace=True)

In [92]:
df = df[df['Distance(mi)'] < 100]

In [93]:
cols = ['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)']

for col in cols:
    if col in df.columns:
        df[col] = df[col].astype('float32')

In [94]:
print("✅ حجم پس از بهینه‌سازی (GB):", df.memory_usage(deep=True).sum() / 1e9)

✅ حجم پس از بهینه‌سازی (GB): 0.968391949


In [95]:
df.head()


,ID,Source,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,...,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Start_Date,Is_Cold,Is_Low_Visibility,Location_Hash
3402762,2680600,0,3,40.108910,-83.092857,40.112061,-83.031868,3.230,1342349,269957,...,False,False,1,1,1,1,1970-01-01,0,0,6193
3402763,2680601,0,2,39.865421,-84.062798,39.865009,-84.048729,0.747,1239529,225926,...,False,False,1,1,1,1,1970-01-01,0,0,1634
3402764,2680602,0,2,39.102661,-84.524681,39.102089,-84.523956,0.055,1211011,225953,...,False,False,1,1,1,0,1970-01-01,0,0,8393
3402765,2680603,0,2,39.101479,-84.523407,39.098412,-84.522408,0.219,1211011,316973,...,False,False,1,1,1,0,1970-01-01,0,0,8393
3402766,2680604,0,2,41.062130,-81.537842,41.062168,-81.535469,0.123,1184327,225965,...,False,False,1,1,0,0,1970-01-01,0,0,1868
